In [21]:
##The purpose of this code is to demonstrate the methodology in cleaning the data in my thesis.
##The sample data used is similar to the actual patent data, with the exception it is a few million rows less than the actual. 

In [48]:
##importing packages
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [49]:
ID = [1, 1, 1, 1, 1, 1, 2, 2, 2, 2]
Country = ["France", "France", "Germany", "Germany", "UK", "UK", "Austria", "Austria", "Germany", "Germany"]
Year = [2011, 2011, 2011, 2011, 2011, 2011, 2015, 2015, 2015, 2015]
Class = ["A", "B", "A", "B", "A", "B", "A", "B", "A", "B"]
Cites = [5, 5, 5, 5, 5, 5, 3, 3, 3, 3]

# Create DataFrame using a dictionary
patent_full = pd.DataFrame({
    'ID': ID,
    'Country': Country,
    'Year': Year,
    'Class': Class,
    'Cites': Cites
})

patent_full
patent_full.describe()

,ID,Country,Year,Class,Cites
0,1,France,2011,A,5
1,1,France,2011,B,5
2,1,Germany,2011,A,5
3,1,Germany,2011,B,5
4,1,UK,2011,A,5
5,1,UK,2011,B,5
6,2,Austria,2015,A,3
7,2,Austria,2015,B,3
8,2,Germany,2015,A,3
9,2,Germany,2015,B,3


,ID,Year,Cites
count,10.000000,10.000000,10.000000
mean,1.400000,2012.600000,4.200000
std,0.516398,2.065591,1.032796
min,1.000000,2011.000000,3.000000
25%,1.000000,2011.000000,3.000000
50%,1.000000,2011.000000,5.000000
75%,2.000000,2015.000000,5.000000
max,2.000000,2015.000000,5.000000


In [50]:
def clean(dataframe):
    dataframe['country_share'] = 0  
    dataframe['class_share'] = 0
    
    for id in dataframe['ID'].unique():
        id_data = dataframe[dataframe['ID'] == id]
        total_countries = id_data.shape[0]  # Total number of occurrences within the unique ID
        
        for country in id_data['Country'].unique():
            country_count = id_data['Country'].value_counts()[country]  # Count of occurrences of the country
            country_share = country_count / total_countries  # Calculate share
            
            # Update 'country_share' column for the specific country and ID
            dataframe.loc[(dataframe['ID'] == id) & (dataframe['Country'] == country), 
                          'country_share'] = country_share
    
    for id in dataframe['ID'].unique():
        id_data = dataframe[dataframe['ID'] == id]
        total_class = id_data.shape[0]  # Total number of occurrences within the unique ID
        
        for class_i in id_data['Class'].unique(): ##class is formally called the cpc code
            class_count = id_data['Class'].value_counts()[class_i]  # Count of occurrences of the country
            class_share = class_count / total_class  # Calculate share
            
            # Update 'country_share' column for the specific country and ID
            dataframe.loc[(dataframe['ID'] == id) & (dataframe['Class'] == class_i), 
                          'class_share'] = class_share
        
            
    return dataframe

In [51]:
clean(patent_full)

,ID,Country,Year,Class,Cites,country_share,class_share
0,1,France,2011,A,5,0.333333,0.5
1,1,France,2011,B,5,0.333333,0.5
2,1,Germany,2011,A,5,0.333333,0.5
3,1,Germany,2011,B,5,0.333333,0.5
4,1,UK,2011,A,5,0.333333,0.5
5,1,UK,2011,B,5,0.333333,0.5
6,2,Austria,2015,A,3,0.500000,0.5
7,2,Austria,2015,B,3,0.500000,0.5
8,2,Germany,2015,A,3,0.500000,0.5
9,2,Germany,2015,B,3,0.500000,0.5


In [52]:
patent_full['country_to_class_share'] = patent_full['country_share']*patent_full['class_share']
patent_full.head()
patent_full.loc[patent_full['ID'] == 1, 'country_to_class_share'].sum() ##country_to_class_share = 1 (success!)

,ID,Country,Year,Class,Cites,country_share,class_share,country_to_class_share
0,1,France,2011,A,5,0.333333,0.5,0.166667
1,1,France,2011,B,5,0.333333,0.5,0.166667
2,1,Germany,2011,A,5,0.333333,0.5,0.166667
3,1,Germany,2011,B,5,0.333333,0.5,0.166667
4,1,UK,2011,A,5,0.333333,0.5,0.166667


0.9999999999999999

In [53]:
##matching country's corporate tax data
Country_Tax = pd.DataFrame({ 
    'Year': [2011, 2015],
    'Austria': [0.45, 0.25],
    'France': [0.4, 0.35],
    'Germany': [0.5, 0.3],
    'UK': [0.3, 0.25]
})

In [54]:
def merge(data1, data2):
    # Iterate over rows in data2
    for index, row in data2.iterrows():
        year = row['Year']
        
        # Iterate over columns (countries) in data2
        for country, tax_value in row.items(): ##country is column, tax value is row
            # Update 'Country_tax' column in data1 if year and country match
            mask = (data1['Year'] == year) & (data1['Country'] == country)
            data1.loc[mask, 'Country_tax'] = tax_value
    
    return data1


In [55]:
patent_full = merge(patent_full, Country_Tax)
patent_full['weighted_cites'] = patent_full['country_to_class_share']*patent_full['Cites']
patent_full

,ID,Country,Year,Class,Cites,country_share,class_share,country_to_class_share,Country_tax,weighted_cites
0,1,France,2011,A,5,0.333333,0.5,0.166667,0.40,0.833333
1,1,France,2011,B,5,0.333333,0.5,0.166667,0.40,0.833333
2,1,Germany,2011,A,5,0.333333,0.5,0.166667,0.50,0.833333
3,1,Germany,2011,B,5,0.333333,0.5,0.166667,0.50,0.833333
4,1,UK,2011,A,5,0.333333,0.5,0.166667,0.30,0.833333
5,1,UK,2011,B,5,0.333333,0.5,0.166667,0.30,0.833333
6,2,Austria,2015,A,3,0.500000,0.5,0.250000,0.25,0.750000
7,2,Austria,2015,B,3,0.500000,0.5,0.250000,0.25,0.750000
8,2,Germany,2015,A,3,0.500000,0.5,0.250000,0.30,0.750000
9,2,Germany,2015,B,3,0.500000,0.5,0.250000,0.30,0.750000


In [56]:
country_patent = patent_full.groupby(['Country', 'Year']).agg({'country_to_class_share': 'sum',
                                     'weighted_cites': 'sum',
                                    'Country_tax': 'first'}).reset_index()


In [57]:
country_patent = country_patent.rename(columns = {"country_to_class_share": "weighted_activity"})
country_patent

,Country,Year,weighted_activity,weighted_cites,Country_tax
0,Austria,2015,0.500000,1.500000,0.25
1,France,2011,0.333333,1.666667,0.40
2,Germany,2011,0.333333,1.666667,0.50
3,Germany,2015,0.500000,1.500000,0.30
4,UK,2011,0.333333,1.666667,0.30
